In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

os.chdir('/content/drive/MyDrive/databowl/')
df = pd.read_csv('2023_tracking.csv')

In [ ]:
play_ids = df['play_id_n'].unique()[2000:4500]
df = df[df['play_id_n'].isin(play_ids)]

In [ ]:
max_targets = df.groupby(['play_id_n', 'frame_id'])['player_to_predict'].sum().max()
print(max_targets)

8


#### 2d Grid of Cords

In [ ]:
def pixel_map(player_data, target_player_ids, max_targets, grid_width=121, grid_height=55, sigma=.8):

    num_channels = 2 + max_targets + 1
    # three channels, one for offense, one for defense, one for ball location, one for player to predict
    pixel_map = np.zeros((num_channels, grid_height, grid_width), dtype=np.float32)

    x_vals = player_data['x'].values
    y_vals = player_data['y'].values

    for i, (x, y) in enumerate(zip(x_vals, y_vals)):
        x_min = max(0, int(np.floor(x - 2*sigma)))
        x_max = min(grid_width, int(np.ceil(x + 2*sigma)) + 1)
        y_min = max(0, int(np.floor(y - 2*sigma)))
        y_max = min(grid_height, int(np.ceil(y + 2*sigma)) + 1)

        for xi in range(x_min, x_max):
            for yi in range(y_min, y_max):
                dist_sq = (xi - x)**2 + (yi - y)**2
                weight = np.exp(-dist_sq / (2 * sigma**2))

                player_id = player_data.iloc[i].get('nfl_id', None)

                if player_id in target_player_ids:
                    target_idx = target_player_ids.index(player_id)
                    if target_idx < max_targets: # to prevent error
                        pixel_map[2 + target_idx, yi, xi] += weight
                elif player_data.iloc[i]['player_side'] == 'Offense':
                    pixel_map[0, yi, xi] += weight
                elif player_data.iloc[i]['player_side'] == 'Defense':
                    pixel_map[1, yi, xi] += weight


    ball_x = player_data['ball_land_x'].iloc[0]
    ball_y = player_data['ball_land_y'].iloc[0]

    ball_x_min = max(0, int(np.floor(ball_x - 2*sigma)))
    ball_x_max = min(grid_width, int(np.ceil(ball_x + 2*sigma)) + 1)
    ball_y_min = max(0, int(np.floor(ball_y - 2*sigma)))
    ball_y_max = min(grid_height, int(np.ceil(ball_y + 2*sigma)) + 1)

    for xi in range(ball_x_min, ball_x_max):
        for yi in range(ball_y_min, ball_y_max):
            dist_sq = (xi - ball_x)**2 + (yi - ball_y)**2
            weight = np.exp(-dist_sq / (2 * sigma**2))
            pixel_map[2 + max_targets, yi, xi] += weight

    return pixel_map

testing one play

In [ ]:
df_play_id = df[df['play_id_n'] == 456]
target_player_ids = df_play_id[df_play_id['player_to_predict'] == True]['nfl_id'].unique().tolist()
df_grids_t = df_play_id.groupby(['play_id_n','frame_id']).apply(
    lambda x: pd.Series({'grid': pixel_map(x, target_player_ids, max_targets)})
)

C:\Users\dalto\AppData\Local\Temp\ipykernel_25552\2474093878.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grids_t = df_play_id.groupby(['play_id_n','frame_id']).apply(


grid for all plays

In [ ]:
df_grids = df.groupby(['play_id_n','frame_id']).apply(
    lambda x: pd.Series({'grid': pixel_map(x,  x[x['player_to_predict'] == True]['nfl_id'].unique().tolist(), max_targets)})
)

/tmp/ipython-input-3746108319.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grids = df.groupby(['play_id_n','frame_id']).apply(


In [ ]:
df_grids = df_grids.reset_index()
df_grids = df_grids.sort_values(['play_id_n', 'frame_id'])

visual test

In [ ]:
sample_grid = df_grids_t['grid'].iloc[30]

# Count how many target player channels have data
num_targets = 0
for i in range(max_targets):
    if sample_grid[2 + i].sum() > 0:
        num_targets += 1

# Create subplots: 2 base channels + ball + target players
total_plots = 3 + num_targets
cols = 4
rows = (total_plots + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(14, 5*rows))
axes = axes.flatten() if total_plots > 1 else [axes]

# Plot offense
axes[0].imshow(sample_grid[0], origin='lower', cmap='Reds')
axes[0].set_title('Offense Players')
axes[0].set_xlabel('X (yards)')
axes[0].set_ylabel('Y (yards)')

# Plot defense
axes[1].imshow(sample_grid[1], origin='lower', cmap='Blues')
axes[1].set_title('Defense Players')
axes[1].set_xlabel('X (yards)')
axes[1].set_ylabel('Y (yards)')

# Plot ball location (at index 2 + max_targets)
axes[2].imshow(sample_grid[2 + max_targets], origin='lower', cmap='Purples')
axes[2].set_title('Ball Landing Location')
axes[2].set_xlabel('X (yards)')
axes[2].set_ylabel('Y (yards)')

# Plot each target player (channels 2 through 2+max_targets-1)
plot_idx = 3
for i in range(max_targets):
    if sample_grid[2 + i].sum() > 0:
        axes[plot_idx].imshow(sample_grid[2 + i], origin='lower', cmap='Greens')
        axes[plot_idx].set_title(f'Target Player {i+1}')
        axes[plot_idx].set_xlabel('X (yards)')
        axes[plot_idx].set_ylabel('Y (yards)')
        plot_idx += 1

for idx in range(total_plots, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
df_grids.to_pickle("full_grids_2500.pkl")

### Transformer

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import gc
from transformers import get_cosine_schedule_with_warmup
df_grids = pd.read_pickle("full_grids_2500.pkl")
torch.manual_seed(26)

#### Global Input Seq Context

In [ ]:
class CNN_DownSample(nn.Module):
    def __init__(self):
        super().__init__()
        # variable based on amount of targets
        input_chan = 2 + max_targets + 1

        # using stride rather than max pooling preforms better as max pooling tends to compress feat. too much.
        self.heatmap_encoder = nn.Sequential(
            nn.Conv2d(in_channels=input_chan, out_channels=16, kernel_size=3, stride = 2, padding=1),
            nn.BatchNorm2d(16), # normalize var and mean
            nn.GELU(), # preforms better on average idk if itll make a difference in this application

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride = 2, padding=1),
            nn.BatchNorm2d(32),
            nn.GELU(),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride = 2, padding=1),
            nn.BatchNorm2d(64),
            nn.GELU()
        )

    def forward(self, x):
        x = self.heatmap_encoder(x)
        return x


attention layer

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, nhead, mask, dropout=0.15):
        super().__init__() # inhert from parent class

        if d_model % nhead != 0:
            raise ValueError(f"d_model ({d_model}) must be divisible by nhead ({nhead})")

        self.d_model = d_model # dimension of model
        self.nhead = nhead # number of attention heads, multi headed
        self.head_dim = d_model // nhead

        # create key query and values
        self.qkv_proj = nn.Linear(d_model, 3 * d_model)
        # learn context as a product of the attention heads
        self.out_proj = nn.Linear(d_model, d_model)
        # dropout as a form of regularzation
        self.dropout = nn.Dropout(dropout)
        # scaling function
        self.scale = self.head_dim ** -0.5

    def forward(self, x, mask=None):
        B, L, _ = x.shape # batch and length

        # create q, k, v values | init just random matrix mults, learned parameter
        qkv = self.qkv_proj(x)

        # split key, query, and value vectors into diff pares
        q, k, v = qkv.chunk(3, dim=-1)

        # transpose the matrix so that batch and nhead are treated as batches and self attention is calculated from there
        q = q.view(B, L, self.nhead, self.head_dim).transpose(1, 2)
        k = k.view(B, L, self.nhead, self.head_dim).transpose(1, 2)
        v = v.view(B, L, self.nhead, self.head_dim).transpose(1, 2)

        # scaled dot product, scale so values arent 0 or 1
        scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale # matrix mult

        # set masked values to -inf so softmax does not "give" attention to them
        if mask is not None:
          if mask.dim() == 2:
            mask = mask.expand(B, self.nhead, L, L)
          elif mask.dim() == 3:
            mask = mask.unsqueeze(1)
            mask = mask.expand(B, self.nhead, L, L)
          elif mask.dim() == 4:
            mask = mask.expand(B, self.nhead, L, L)

          scores = scores.masked_fill(mask == 0, -1e9)


        # softmax to give attention weights to each token
        attn_weights = torch.softmax(scores, dim=-1)

        # drop some weights
        attn_weights = self.dropout(attn_weights)

        # context vector for a given input sequence
        context = torch.matmul(attn_weights, v)

        # transpose so the matrix is in the correct size to be concatinated
        context = context.transpose(1, 2).contiguous().view(B, L, self.d_model)

        # "combine" the outputs from the head to one general vector
        output = self.out_proj(context)

        return output

transformer block

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, nhead=4, mask=None, dropout=0.15):
        super(TransformerBlock, self).__init__()
        # self attention class definied above
        self.self_attn = MultiHeadAttention(d_model=d_model, nhead=nhead, dropout=dropout, mask=mask)

        # feed forward network for each token
        self.feed_forward = nn.Sequential(
            nn.Linear(64, 128),
            nn.GELU(),
            nn.Dropout(dropout), # to combat overfitting
            nn.Linear(128, 64)
        )

        # normilzations so values are between 0-1, learned gamma and beta parameters
        # to shift center and var for values.
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # standard dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        residual = x
        # normalized pre attention layer, gradients flow black directly without the normalizing effecting x values
        norm_x = self.norm1(x)
        # self attention
        attn_output = self.self_attn(norm_x, mask)
        # adding residual back to self attention
        x = residual + self.dropout(attn_output)

        residual = x
        # normalize values
        # we do so because over the amount of layers scale can get distorted, lead to super big or small values
        norm_x = self.norm2(x)
        # basic fcn
        ff_output = self.feed_forward(norm_x)
        # adding residual back so that the gradient can flow directly back.
        # adds a 1 + terms to gradients, helps solve the vanishing gradients problem
        x = residual + self.dropout(ff_output)

        return x

In [ ]:
# use sinusoidal functions, simpler than learned values and generalizes better to unseen parameters
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, dropout, max_length=150):
        super(PositionalEncoding, self).__init__()
        # droput
        self.dropout = nn.Dropout(p=dropout)

        # create matrix
        pe = torch.zeros(max_length, embed_size)

        # position tensor shape
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)

        # div_term tensor shape
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-torch.log(torch.tensor(10000.0)) / embed_size))

        # apply sin to even indices
        pe[:, 0::2] = torch.sin(position * div_term)
        # apply cos to odd indices
        if embed_size % 2 == 1:
            pe[:, 1::2] = torch.cos(position * div_term[:-1])
        else:
            pe[:, 1::2] = torch.cos(position * div_term)

        # register as buffer so it moves with model to device
        self.register_buffer('pe', pe)

    def forward(self, x):
        pe_slice = self.pe[:x.size(1), :].to(x.device)
        x = x + pe_slice
        return self.dropout(x)

class TransEncoder(nn.Module):
    def __init__(self, input_dim, embed_size, num_layers, device, dropout, mask, max_length):
        super(TransEncoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        # learned matrix projection
        self.input_projection = nn.Linear(input_dim, embed_size)
        # postional encoding
        self.position_encoding = PositionalEncoding(embed_size, dropout, max_length)
        # layers of model, just stacked encoding layer
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    mask=mask,
                    nhead=4, # number of attention heads
                    dropout=dropout
                )
                for _ in range(num_layers)
            ]
        )
        # normalize after attention
        self.norm = nn.LayerNorm(embed_size)

    def forward(self, x, mask):
        # input layer matrix mult
        projected_input = self.input_projection(x)
        # position encodings
        out = self.position_encoding(projected_input)
        # mask to correct dim
        mask = mask.unsqueeze(1).unsqueeze(2)
        # pass through transformer block
        for layer in self.layers:
            out = layer(out, mask)
        # normalize gradients
        out = self.norm(out)

        return out

##### Sequence

In [ ]:
# prevent lookahead
def create_causal_mask(seq_len, device):
    mask = torch.tril(torch.ones(seq_len, seq_len, device=device))
    return mask.bool()

# seq mask to deal with padded values
def seq_maks(input_lengths, max_input, device):
    batch_size = len(input_lengths)
    input_mask = torch.zeros(batch_size, max_input, device=device, dtype=torch.bool)
    for i, length in enumerate(input_lengths):
        input_mask[i, :length] = True

# loss with mask
def mse_with_length_mask(predictions, targets, combined_mask):
    mse = (predictions - targets) ** 2
    masked_mse = mse * combined_mask

    valid_elements = combined_mask.sum()
    if valid_elements > 0:
        return masked_mse.sum() / valid_elements
    else:
        return torch.tensor(0.0, device=predictions.device)

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, dropout, embedding, nhead):
        super().__init__()
        # attention layers
        self.attention_self = MultiHeadAttention(d_model=embedding, nhead=4, mask=None, dropout=dropout)

        # cross atten, query values, info
        self.cross_q_proj = nn.Linear(embedding, embedding)
        self.cross_k_proj = nn.Linear(embedding, embedding)
        self.cross_v_proj =  nn.Linear(embedding, embedding)
        self.cross_out_proj = nn.Linear(embedding, embedding)

        self.nhead = nhead
        self.head_dim = embedding // nhead
        self.scale = self.head_dim ** -0.5 # 1/sqrt(dk)

        # layer normal
        self.norm1 = nn.LayerNorm(embedding)
        self.norm2 = nn.LayerNorm(embedding)
        self.norm3 = nn.LayerNorm(embedding)

        # droput
        self.dropout = nn.Dropout(dropout)

        # fcnn
        self.fcnn = nn.Sequential(
            nn.Linear(embedding, embedding*2),
            nn.GELU(),
            nn.Dropout(0.15),
            nn.Linear(embedding*2, embedding)
        )
    # decoder forward pass
    def forward(self, decoder_input, encoded_context, target_mask = None, casual_mask=None):

        # self attention amoung decoder
        residual = decoder_input
        norm_x = self.norm1(decoder_input)
        self_attn = self.attention_self(norm_x, casual_mask)
        decoder_input = residual + self.dropout(self_attn)

        # cross attention to encoder
        norm_x = self.norm2(decoder_input)
        cross_atn = self.encoder_cross_attention(norm_x, encoded_context)
        # dropout, also cant do inplace ops bc of backprop
        decoder_input = decoder_input + self.dropout(cross_atn)

        # fcnn predictions
        norm_x = self.norm3(decoder_input)
        ffcn = self.fcnn(norm_x)
        out = decoder_input + self.dropout(ffcn)

        return out

    def encoder_cross_attention(self, query, key_value):
        B, L_q, _ = query.shape # decoder input
        B, L_kv, _ = key_value.shape # encoder output

        q = self.cross_q_proj(query)
        k = self.cross_k_proj(key_value)
        v = self.cross_v_proj(key_value)

        q = q.view(B, L_q, self.nhead, self.head_dim).transpose(1, 2)
        k = k.view(B, L_kv, self.nhead, self.head_dim).transpose(1, 2)
        v = v.view(B, L_kv, self.nhead, self.head_dim).transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale

        attn_weights = torch.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context = torch.matmul(attn_weights, v)
        context = context.transpose(1, 2).contiguous().view(B, L_q, -1)

        output = self.cross_out_proj(context)

        return output

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, dropout, max_length=150):
        super(PositionalEncoding, self).__init__()
        # droput
        self.dropout = nn.Dropout(p=dropout)

        # create matrix
        pe = torch.zeros(max_length, embed_size)

        # position tensor shape
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)

        # div_term tensor shape
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-torch.log(torch.tensor(10000.0)) / embed_size))

        # apply sin to even indices
        pe[:, 0::2] = torch.sin(position * div_term)
        # apply cos to odd indices
        if embed_size % 2 == 1:
            pe[:, 1::2] = torch.cos(position * div_term[:-1])
        else:
            pe[:, 1::2] = torch.cos(position * div_term)

        # register as buffer so it moves with model to device
        self.register_buffer('pe', pe)

    def forward(self, x):
        pe_slice = self.pe[:x.size(1), :].to(x.device)
        x = x + pe_slice
        return self.dropout(x)

In [ ]:
class TransDecoder(nn.Module):
    def __init__(self, target_mask, embedding, dropout, nhead, layers, max_targets, max_step_change, max_seq_len):
        super(TransDecoder, self).__init__()
        self.max_targets = max_targets
        self.max_seq_len = max_seq_len
        self.embedding = embedding
        self.max_step = max_step_change

        # project 2d cords to embedding space
        self.positon_projection = nn.Linear(2, embedding)

        # project outputs back to 2d space
        self.output_projection = nn.Linear(embedding, 2)

        # postional embeddings
        self.pos_embed = PositionalEncoding(embed_size=embedding, dropout=0.15, max_length=150)

        # decoder layers
        self.layers = nn.ModuleList([
            DecoderLayer(embedding=embedding, dropout=dropout, nhead=nhead)
            for _ in range(layers)
        ])

        # normalization
        self.norm = nn.LayerNorm(embedding)

    def forward(self, encoded_context, start_positons, target_mask, max_step_change, output_lengths):
        # batch size and device
        batch_size = encoded_context.shape[0]
        device = encoded_context.device

        # max output length
        max_output_len = output_lengths.max().item()

        # output tensor
        all_outputs = torch.zeros(batch_size, self.max_targets, max_output_len, 2, device=device)

        # decoder seq
        decoder_sequence = [[] for _ in range(batch_size)]
        current_postions = start_positons.clone()

        # output sequence
        for step in range(max_output_len):
            step_mask = step < output_lengths
            active_batch_size = step_mask.sum().item()

            if active_batch_size == 0:
                break

            # project pos to embedding space
            pos_embeds = self.positon_projection(current_postions)

            # add positional context
            pos_encoding = self.pos_embed.pe[step, :self.embedding].unsqueeze(0).unsqueeze(0)
            # Replace inplace addition with out-of-place addition
            pos_input = pos_embeds + pos_encoding

            # add sequence dim back
            pos_input = pos_input.unsqueeze(2)

            # decoder seq
            for batch_idx in range(batch_size):
                if step_mask[batch_idx]:
                    decoder_sequence[batch_idx].append(pos_input[batch_idx])

            if step == 0:
                # if first step use init pos
                decoder_input = pos_input
                casual_mask = None
            else:
                # decoder inputs
                batch_decoder_inputs = []

                # get decoder seq for values
                for batch_idx in range(batch_size):
                    if step_mask[batch_idx]:
                        sample_sequence = torch.cat(decoder_sequence[batch_idx], dim = 1)
                        batch_decoder_inputs.append(sample_sequence)
                    else: # if padded value etc.
                        dummy_input = torch.zeros(self.max_targets, step+1, self.embedding, device=device)
                        batch_decoder_inputs.append(dummy_input)

                # all samples for decoder input
                decoder_input = torch.stack(batch_decoder_inputs)

                # casual mask
                seq_len = step + 1
                casual_mask = create_causal_mask(seq_len, device)


            # transformer layers
            batch_size_curr, max_targets_curr, seq_len_curr, embed_dim_curr = decoder_input.shape
            decoder_input_reshaped = decoder_input.view(batch_size_curr * max_targets_curr, seq_len_curr, embed_dim_curr)

            # decoder layer
            decoded = decoder_input_reshaped

            # masking for future seq
            if casual_mask is not None:
               casual_mask_expanded = casual_mask.unsqueeze(0).expand(batch_size_curr * max_targets_curr, seq_len_curr, seq_len_curr)
            else:
               casual_mask_expanded = None


            for layer in self.layers:
                decoded = layer(decoded, encoded_context.repeat_interleave(max_targets_curr, dim=0), target_mask, casual_mask_expanded)

            # normalize gradients
            decoded = self.norm(decoded)

            # reshape and project back to cords
            decoded = decoded.view(batch_size_curr, max_targets_curr, seq_len_curr, embed_dim_curr)
            predictions = self.output_projection(decoded[:, :, -1, :])

            clamp_pred = predictions

            if self.max_step is not None and self.max_step > 0:
              # change in positon
              delta = predictions - current_postions
              # distance between
              dist = torch.norm(delta, p=2, dim=-1)
              # scale preds
              scale_factor = torch.clamp(dist / (self.max_step + 1e-9), max=1).unsqueeze(-1)
              # clamped preds
              clamp_pred = current_postions + scale_factor * delta

            for batch_idx in range(batch_size):
                if step_mask[batch_idx]:
                    all_outputs[batch_idx, :, step, :] = clamp_pred[batch_idx]

            # stack all predictions
            current_postions = clamp_pred.clone()

        # target mask
        target_mask_exp = target_mask.unsqueeze(-1).unsqueeze(-1)
        all_outputs = all_outputs * target_mask_exp

        return all_outputs

In [ ]:
def train_seq_2_seq(df_grids, max_targets, max_input, max_output):
    # arrays and postions
    sequences = []
    targets = []
    masks = []
    start_positions = []
    input_lengths = []
    output_lengths = []

    # get all offensive players
    player_to_predict = df[df['player_to_predict'] == True].groupby('play_id_n')['nfl_id'].unique()

    # get players to predict postions
    for play_id in df_grids['play_id_n'].unique():
        play_data = df_grids[df_grids['play_id_n'] == play_id].sort_values('frame_id')

    # loop through every play
    for play_id in df_grids['play_id_n'].unique():
        # data for play
        play_data = df_grids[df_grids['play_id_n'] == play_id].sort_values('frame_id')
        df_data = df[df['play_id_n'] == play_id]
        # frames of input and output
        total_frames = play_data['frame_id'].max()
        output_frames = int(df_data['num_frames_output'].max())

        # get players to predict ids
        players = player_to_predict[play_id]

        # only take max targets
        players = players[:max_targets]
        num_receivers = len(players)

        # mask of players
        mask = torch.zeros(max_targets)
        mask[:num_receivers] = 1

        # sequence lengths
        input_seq_len = total_frames - output_frames
        output_seq_len = output_frames

        # input sequence
        input_grids = [torch.from_numpy(grid).float() for grid in play_data['grid'].iloc[:input_seq_len]]
        sequence = torch.stack(input_grids, dim=0)

        # add noise for regularization
        noise = torch.randn_like(sequence) * 0.0001
        input_sequence = sequence + noise

        # pad sequence to max_size if necessary
        if input_sequence.shape[0] < max_input:
            padding_size = max_input - input_sequence.shape[0]
            padding = torch.zeros(padding_size, *input_sequence.shape[1:])
            input_sequence = torch.cat([input_sequence, padding], dim=0)
        elif input_sequence.shape[0] > max_input:
            input_sequence = input_sequence[:max_input]

        # last frame of input sequence
        start_pos = torch.zeros(max_targets, 2)
        last_frame_id = play_data.iloc[input_seq_len - 1]['frame_id']
        last_frame_data = df[(df['play_id_n'] == play_id) & (df['frame_id'] == last_frame_id)]

        # get cords for the last frame of the input seq
        for i, receiver_id in enumerate(players):
            receiver_data = last_frame_data[last_frame_data['nfl_id'] == receiver_id]
            if not receiver_data.empty:
                x = float(receiver_data['x'].iloc[0]) / 120
                y = float(receiver_data['y'].iloc[0]) / 53.3
                start_pos[i] = torch.tensor([x, y])

        # target sequence
        target_positions = torch.zeros(max_targets, max_output, 2)

        # append postions for every targeted reciver
        for step in range(output_seq_len):
            frame_idx = input_seq_len + step
            if frame_idx < len(play_data):
                target_frame = play_data.iloc[frame_idx]['frame_id']
                last_frame_data = df[(df['play_id_n'] == play_id) & (df['frame_id'] == target_frame)]

            for i, receiver_id in enumerate(players):
                receiver_data = last_frame_data[last_frame_data['nfl_id'] == receiver_id]
                if not receiver_data.empty:
                    x = float(receiver_data['x'].iloc[0]) / 120
                    y = float(receiver_data['y'].iloc[0]) / 53.3
                    target_positions[i, step] = torch.tensor([x, y])

        # if there is targets
        if num_receivers > 0:
            sequences.append(input_sequence)
            targets.append(target_positions)
            masks.append(mask)
            start_positions.append(start_pos)
            input_lengths.append(input_seq_len)
            output_lengths.append(output_seq_len)

    if len(sequences) == 0:
        return None, None, None, None, None, None

    return (torch.stack(sequences, dim=0),
            torch.stack(targets, dim=0),
            torch.stack(masks, dim=0),
            torch.stack(start_positions, dim=0),
            torch.tensor(input_lengths),
            torch.tensor(output_lengths))

In [ ]:
class MaskedSequenceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, predictions, targets, target_mask):
        mask = target_mask.unsqueeze(-1).unsqueeze(-1)
        mask = mask.expand_as(predictions)

        mse = (predictions - targets) ** 2
        # dont consider error on padding, masked tokens etc
        masked_mse = mse * mask

        # normalize for masked elements
        valid = mask.sum()
        if valid > 0:
            return masked_mse.sum() / valid
        else:
            return torch.tensor(0.0, device=predictions.device)


In [ ]:
class DJMooreSeq(nn.Module):
    def __init__(self, embed_size, encoder_layers, decoder_layers,
                 max_targets, dropout, nheads, max_step, dev='cuda') -> None:
        super().__init__()

        # general vars
        self.embedding_size = embed_size
        self.max_targets = max_targets
        self.device = dev

        # context cnn
        self.context_cnn = CNN_DownSample()
        context_cnn_output = 64* 7 * 16

        # transformer encoder
        self.encoder = TransEncoder(input_dim=context_cnn_output,
                                    embed_size=embed_size,
                                    num_layers=encoder_layers,
                                    device=dev,
                                    mask=None,
                                    dropout=dropout,
                                    max_length=150)

        # decoder
        self.decoder = TransDecoder(target_mask=None,
                                    embedding=embed_size,
                                    dropout=dropout,
                                    nhead=nheads,
                                    layers=decoder_layers,
                                    max_targets=max_targets,
                                    max_step_change=max_step,
                                    max_seq_len=max_output)

    def forward(self, heatmap_sequence, start_pos, target_mask, future_steps, input_lengths):
        # derive batch size, length of transformer output
        batch_size, seq_len = heatmap_sequence.shape[:2]
        # cnn features
        cnn_features = []
        for t in range(seq_len):
            frame = heatmap_sequence[:,t]
            features = self.context_cnn(frame)
            features = features.flatten(1)
            cnn_features.append(features)

        # stack and encode
        sequence_feat = torch.stack(cnn_features, dim=1)

        # encoder mask based on input seq
        max_input_len = seq_len
        encoder_mask = torch.zeros(batch_size, max_input_len, device=heatmap_sequence.device, dtype=torch.bool)
        for i, length in enumerate(input_lengths):
            encoder_mask[i, :length] = True

        # context
        encoded_context = self.encoder(sequence_feat, encoder_mask)
        # output predictions
        predictions = self.decoder(encoded_context, start_pos, target_mask, future_steps, input_lengths)

        return predictions

##### training

In [ ]:
def train(model, train_loader, val_loader, optimizer, scheduler, epochs):
  # loss
  train_losses = []
  val_losses = []
  # early stopping
  best_loss = np.inf
  early_stop_rounds = 0

  for epoch in range(epochs):
    # train mode, loss and batches
    model.train()
    epoch_loss = 0
    batches = 0

    for batch_sequence, batch_targets, batch_masks, batch_start_pos, batch_input_lengths, batch_output_lengths in train_loader:

        # move all items to gpu
        batch_sequence = batch_sequence.to('cuda')
        batch_targets = batch_targets.to('cuda')
        batch_masks = batch_masks.to('cuda')
        batch_start_pos = batch_start_pos.to('cuda')
        batch_input_lengths = batch_input_lengths.to('cuda')
        batch_output_lengths = batch_output_lengths.to('cuda')

        # zero the gradients
        optimizer.zero_grad()

        # forward pass
        predictions = model(batch_sequence, batch_start_pos, batch_masks,
                            batch_output_lengths, batch_input_lengths)

        # masks for loss function
        _, _, pred_len, _ = predictions.shape
        batch_size, _, targ_len, _ = batch_targets.shape
        max_seq_len = min(pred_len, targ_len)

        # consitent sizing
        predictions_sliced = predictions[:, :, :max_seq_len, :]
        batch_targets_sliced = batch_targets[:, :, :max_seq_len, :]
        batch_masks_sliced = batch_masks[:, :max_seq_len]

        # slice preds to correct len
        len_mask = torch.zeros(batch_size, max_seq_len, device=predictions_sliced.device, dtype=torch.bool)

        for i, length in enumerate(batch_output_lengths):
            len_mask[i, :length] = True

        # masks
        target_mask_expanded = batch_masks_sliced.unsqueeze(-1).unsqueeze(-1)
        length_mask_expanded = len_mask.unsqueeze(1).unsqueeze(-1)
        combined_mask = target_mask_expanded * length_mask_expanded

        # train loss
        loss = mse_with_length_mask(predictions_sliced, batch_targets_sliced, combined_mask.expand_as(predictions_sliced))

        # backprop
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # epoch loss
        epoch_loss += loss.item() * batch_sequence.size(0)
        batches += batch_sequence.size(0)

    # training losses
    avg_loss = epoch_loss / batches
    train_losses.append(avg_loss)

    # val set losses
    model.eval()
    val_epoch_loss = 0
    val_batches = 0

    with torch.no_grad():
        for val_sequence, val_targets, val_masks, val_start_pos, val_input_lengths, val_output_lengths in val_loader:
            # to cuda
            val_sequence = val_sequence.to('cuda')
            val_targets = val_targets.to('cuda')
            val_masks = val_masks.to('cuda')
            val_start_pos = val_start_pos.to('cuda')
            val_input_lengths = val_input_lengths.to('cuda')
            val_output_lengths = val_output_lengths.to('cuda')

            # predictions and loss on val set
            val_predictions = model(val_sequence, val_start_pos, val_masks, val_output_lengths, val_input_lengths)

            # size
            _, _, pred_len, _ = val_predictions.shape
            batch_size, _, targ_len, _ = val_targets.shape
            max_seq_len = min(pred_len, targ_len)

            # mask for sequences
            val_predictions_sliced = val_predictions[:, :, :max_seq_len, :]
            val_targ_sliced = val_targets[:, :, :max_seq_len, :]
            val_mask_sliced = val_masks[:, :max_seq_len]

            length_mask = torch.zeros(batch_size, max_seq_len, device=val_predictions.device, dtype=torch.bool)
            for i, length in enumerate(val_output_lengths):
                length_mask[i, :length] = True

            # masking
            target_mask_expanded = val_mask_sliced.unsqueeze(-1).unsqueeze(-1)
            length_mask_expanded = length_mask.unsqueeze(1).unsqueeze(-1)
            combined_mask = target_mask_expanded * length_mask_expanded

            # Slice targets to match the sequence length of predictions
            val_targets_sliced = val_targets[:, :, :max_seq_len, :]

            # validtion losses
            val_loss = mse_with_length_mask(val_predictions_sliced, val_targets_sliced, combined_mask.expand_as(val_predictions_sliced))
            val_epoch_loss += val_loss.item() * val_sequence.size(0)
            val_batches += val_sequence.size(0)

    # val set losses
    val_epoch_loss = val_epoch_loss / val_batches
    val_losses.append(val_epoch_loss)

    if scheduler is not None:
        scheduler.step(val_epoch_loss)

    print(f'val_loss {val_epoch_loss}, train_loss {avg_loss}')

    # early stopping check
    if val_epoch_loss < best_loss:
        best_loss = val_epoch_loss
        early_stop_rounds = 0
        torch.save(model.state_dict(), f"2500_4_4_4_model{epoch}.pth")
    else:
        early_stop_rounds += 1

    if early_stop_rounds > 20:
        print('early stopping')
        return train_losses, val_losses

  return train_losses, val_losses

In [ ]:
max_targets = df.groupby(['play_id_n', 'frame_id'])['player_to_predict'].sum().max()
max_input = (df['frame_id'] - df['num_frames_output']).max()
max_output = (df['num_frames_output']).max()

36.0
106.0


In [ ]:
def calculate_max_delta(df):
    max_delta = 0.0
    for play_id in df['play_id_n'].unique():
        play_data = df[df['play_id_n'] == play_id].sort_values('frame_id')

        for nfl_id in play_data['nfl_id'].unique():
            player_data = play_data[play_data['nfl_id'] == nfl_id].sort_values('frame_id')
            if len(player_data) > 1:
                deltas = np.sqrt(np.diff(player_data['x'].values)**2 + np.diff(player_data['y'].values)**2)
                if len(deltas) > 0:
                    current_max_delta = np.max(deltas)
                    max_delta = max(max_delta, current_max_delta)

    return max_delta

max_delta_value = calculate_max_delta(df)
print(f"max_delta {max_delta_value}")

max_delta 1.4173566946961516


In [ ]:
# create dataset instance by instance rather than all at once
class NFLSequenceDataset(Dataset):
    def __init__(self, df_grids, df_tracking, max_targets, max_input, max_output, split_indices=None):
        self.df_grids = df_grids
        self.df_tracking = df_tracking
        self.max_targets = max_targets
        self.max_input = max_input
        self.max_output = max_output

        # playe_ids
        self.play_ids = df_grids['play_id_n'].unique()
        if split_indices is not None:
            self.play_ids = self.play_ids[split_indices]

        # players to predict for each play
        self.player_to_predict = df_tracking[df_tracking['player_to_predict'] == True].groupby('play_id_n')['nfl_id'].unique()

    def __len__(self):
        # how many plays
        return len(self.play_ids)

    def __getitem__(self, idx):
        play_id = self.play_ids[idx]

        # data for a given play
        play_data = self.df_grids[self.df_grids['play_id_n'] == play_id].sort_values('frame_id')
        df_data = self.df_tracking[self.df_tracking['play_id_n'] == play_id]

        # seq lengths
        total_frames = play_data['frame_id'].max()
        output_frames = int(df_data['num_frames_output'].max())
        input_seq_len = total_frames - output_frames
        output_seq_len = output_frames

        # target play ids
        if play_id in self.player_to_predict:
            players = self.player_to_predict[play_id][:self.max_targets]
        else:
            players = []

        num_receivers = len(players)

        # mask for less than max target
        mask = torch.zeros(self.max_targets)
        mask[:num_receivers] = 1

        # input seq
        input_grids = []
        for i in range(min(input_seq_len, self.max_input)):
            grid = torch.from_numpy(play_data['grid'].iloc[i]).float()
            noise = torch.randn_like(grid) * 0.0001
            input_grids.append(grid + noise)

        # pad or truncate as needed
        while len(input_grids) < self.max_input:
            input_grids.append(torch.zeros_like(input_grids[0]))
        input_sequence = torch.stack(input_grids[:self.max_input], dim=0)

        # start positons
        start_pos = torch.zeros(self.max_targets, 2)
        last_frame_id = input_seq_len
        last_frame_data = self.df_tracking[
            (self.df_tracking['play_id_n'] == play_id) &
            (self.df_tracking['frame_id'] == last_frame_id)
        ]

        # init postions fo start plyer
        for i, receiver_id in enumerate(players):
            receiver_data = last_frame_data[last_frame_data['nfl_id'] == receiver_id]
            if not receiver_data.empty:
                x = float(receiver_data['x'].iloc[0]) / 120
                y = float(receiver_data['y'].iloc[0]) / 53.3
                start_pos[i] = torch.tensor([x, y])

        # target seq
        output_len = min(output_seq_len, self.max_output)
        target_positions = torch.zeros(self.max_targets, self.max_output, 2)

        # target postions
        for step in range(output_len):
            frame_idx = input_seq_len + step
            if frame_idx < len(play_data):
                target_frame = play_data.iloc[frame_idx]['frame_id']
                target_frame_data = self.df_tracking[
                    (self.df_tracking['play_id_n'] == play_id) &
                    (self.df_tracking['frame_id'] == target_frame)
                ]

                for i, receiver_id in enumerate(players):
                    receiver_data = target_frame_data[target_frame_data['nfl_id'] == receiver_id]
                    if not receiver_data.empty:
                        x = float(receiver_data['x'].iloc[0]) / 120
                        y = float(receiver_data['y'].iloc[0]) / 53.3
                        target_positions[i, step] = torch.tensor([x, y])

        final_output_len = min(output_len, self.max_output)

        return (
            input_sequence,
            target_positions,
            mask,
            start_pos,
            torch.tensor(min(input_seq_len, self.max_input)),
            torch.tensor(final_output_len)
        )

In [ ]:
# split by indexs
play_indices = np.arange(len(df_grids['play_id_n'].unique()))
train_idx, test_idx = train_test_split(play_indices, test_size=0.3, random_state=26)
test_idx, val_idx = train_test_split(test_idx, test_size=0.7, random_state=26)

# create the datasets dynamically
train_dataset = NFLSequenceDataset(df_grids, df, max_targets, int(max_input), int(max_output), train_idx)
val_dataset = NFLSequenceDataset(df_grids, df, max_targets, int(max_input), int(max_output), val_idx)
test_dataset = NFLSequenceDataset(df_grids, df, max_targets, int(max_input), int(max_output), test_idx)

# create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True, num_workers=3)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=3)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=3)

In [ ]:
# clean envi
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# model
model = DJMooreSeq(embed_size=64, encoder_layers=4, decoder_layers=4, max_targets=max_targets, max_step=1.4, dropout=0.1, nheads=4,dev='cuda').to('cuda')

# epochs
epochs = 1000

# loss, opti, schedu
loss_func = MaskedSequenceLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.001)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=5,  num_training_steps=epochs)

# train
train_losses, val_losses = train(model, train_loader, val_loader, optimizer, scheduler, epochs)

val_loss 0.7201902161325727, train_loss 0.5552555061535004


/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


val_loss 0.021934190418216443, train_loss 0.04879347829857346
val_loss 0.02249508662326705, train_loss 0.014003727700424577
val_loss 0.02222924214388643, train_loss 0.01345626159075545
val_loss 0.023203048568573737, train_loss 0.013043084339913698
val_loss 0.02272804965752931, train_loss 0.012274965993128717
val_loss 0.022206641427640405, train_loss 0.012353327447856102
val_loss 0.023770719586561125, train_loss 0.011801826648004011
val_loss 0.023511547085252545, train_loss 0.011968348401709707
val_loss 0.022701242178501117, train_loss 0.011412625945463708
val_loss 0.020144831647298165, train_loss 0.011385730192184858
val_loss 0.02019090379721352, train_loss 0.010782812869415508
val_loss 0.016355494230630853, train_loss 0.011013833922334015
val_loss 0.019603052658161948, train_loss 0.010741839011268127
val_loss 0.020300097972864195, train_loss 0.01057142527654357
val_loss 0.01861149156111337, train_loss 0.010205569596952596
val_loss 0.020170228707471063, train_loss 0.009858721778557183


In [ ]:
def test(model, test_loader):
    model.eval()
    total_loss = 0
    total_batches = 0

    with torch.no_grad():
        for batch_sequence, batch_targets, batch_masks, batch_start_pos, batch_input_lengths, batch_output_lengths in test_loader:
            batch_sequence = batch_sequence.to('cuda')
            batch_targets = batch_targets.to('cuda')
            batch_masks = batch_masks.to('cuda')
            batch_start_pos = batch_start_pos.to('cuda')
            batch_input_lengths = batch_input_lengths.to('cuda')
            batch_output_lengths = batch_output_lengths.to('cuda')

            predictions = model(batch_sequence, batch_start_pos, batch_masks,
                                batch_output_lengths, batch_input_lengths)

            batch_size, _, max_seq_len, _ = predictions.shape
            len_mask = torch.zeros(batch_size, max_seq_len, device=predictions.device, dtype=torch.bool)
            for i, length in enumerate(batch_output_lengths):
                len_mask[i, :length] = True

            target_mask_expanded = batch_masks.unsqueeze(-1).unsqueeze(-1)
            length_mask_expanded = len_mask.unsqueeze(1).unsqueeze(-1)
            combined_mask = target_mask_expanded * length_mask_expanded

            batch_targets_sliced = batch_targets[:, :, :max_seq_len, :]
            loss = mse_with_length_mask(predictions, batch_targets_sliced, combined_mask.expand_as(predictions))

            total_loss += loss.item() * batch_sequence.size(0)
            total_batches += batch_sequence.size(0)

    avg_loss = total_loss / total_batches
    return avg_loss


test_loss = test(model, test_loader)
print(f'Test Loss: {test_loss}')

RuntimeError: The size of tensor a (43) must match the size of tensor b (36) at non-singleton dimension 2

In [ ]:
def visualize_predictions(model, data_loader, play_idx=0, target_player_idx=0):

    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for batch_sequence, batch_targets, batch_masks, batch_start_pos, batch_input_lengths, batch_output_lengths in data_loader:
            batch_sequence = batch_sequence.to('cuda')
            batch_targets = batch_targets.to('cuda')
            batch_masks = batch_masks.to('cuda')
            batch_start_pos = batch_start_pos.to('cuda')
            batch_input_lengths = batch_input_lengths.to('cuda')
            batch_output_lengths = batch_output_lengths.to('cuda')


            predictions = model(batch_sequence, batch_start_pos, batch_masks,
                                batch_output_lengths, batch_input_lengths)

            predicted_trajectory = predictions[play_idx, target_player_idx, :, :].cpu().numpy()
            actual_trajectory = batch_targets[play_idx, target_player_idx, :, :].cpu().numpy()
            mask = batch_masks[play_idx, target_player_idx].cpu().numpy()
            output_length = batch_output_lengths[play_idx].cpu().numpy()


            predicted_trajectory = predicted_trajectory[:output_length]
            actual_trajectory = actual_trajectory[:output_length]

            predicted_trajectory[:, 0] *= 120
            predicted_trajectory[:, 1] *= 53.3
            actual_trajectory[:, 0] *= 120
            actual_trajectory[:, 1] *= 53.3

            plt.figure(figsize=(12, 6))
            plt.plot(actual_trajectory[:, 0], actual_trajectory[:, 1], marker='o', linestyle='-', color='blue', label='Actual')
            plt.plot(predicted_trajectory[:, 0], predicted_trajectory[:, 1], marker='x', linestyle='--', color='red', label='Predicted')
            plt.xlabel('X Coordinate (yards)')
            plt.ylabel('Y Coordinate (yards)')
            plt.title(f'Predicted vs Actual Trajectory for Play {play_idx}, Target Player {target_player_idx}')
            plt.legend()
            plt.grid(True)
            plt.show()

            break

visualize_predictions(model, test_loader, play_idx=5, target_player_idx=1)

In [ ]:
torch.save(model.state_dict(), "2500_4_4_4_model.pth")